# DDX water dataset

In [1]:
from qcportal import PortalClient
from qcelemental.models.results import WavefunctionProtocolEnum
from openff.qcsubmit.results import OptimizationResultCollection,BasicResultCollection
from openff.qcsubmit.datasets import BasicDataset
from openff.qcsubmit.results.filters import ConnectivityFilter, ConformerRMSDFilter
from openff.qcsubmit.factories import BasicDatasetFactory
from openff.qcsubmit.common_structures import Metadata, QCSpec

# Load datasets prepared in `subsample_esp_ds.ipynb`

In [2]:
client = PortalClient("https://api.qcarchive.molssi.org:443/")

In [3]:
br_dataset = BasicResultCollection.parse_file('br_subsample_filtered.json')

In [4]:
esp50k_dataset = BasicResultCollection.parse_file('esp_subsample_filtered.json')

In [5]:
i_dataset = BasicResultCollection.parse_file('i_subsample_filtered.json')

In [6]:
# These were already filtered earlier
br_records = br_dataset.to_records()
esp50k_records = esp50k_dataset.to_records()
i_records = i_dataset.to_records()

# Set up single points

In [7]:
from openff.qcsubmit.common_structures import SCFProperties, Metadata

# want a finer grid for wb97 functionals
keywords = {
    "dft_spherical_points": 590,
    "dft_radial_points": 99
}

# let's calculate a bunch of stuff while we're at it; in case we decide to train on any of this.
properties = [
        SCFProperties.Dipole,
        SCFProperties.Quadrupole,
        SCFProperties.LowdinCharges,
        SCFProperties.MullikenCharges,
        SCFProperties.MBISCharges,
        SCFProperties.MayerIndices,
        SCFProperties.WibergLowdinIndices,
        SCFProperties.DipolePolarizabilities,
    ]


In [8]:
from openff.qcsubmit.common_structures import DDXSettings
# Solvent specs
ddx_spec = DDXSettings(
    ddx_solvent_epsilon=78.4, 
    ddx_radii_scaling = 1.1, # default but wanted to specify explicitly
    ddx_radii_set = 'uff', # default but wanted to specify explicitly 
    ddx_model = 'pcm'
)

ddx_qc_spec = QCSpec(
            program="psi4",
            method="pbe0",
            basis="def2-TZVPPD",
            spec_name="pbe0/def2-TZVPPD/ddx-water",
            spec_description=(
                "PBE0 functional with def2-TZVPPD basis set and DDX water (eps = 78.4) using UFF radii and 1.1x scaling factor"
            ),
            # store_wavefunction=WavefunctionProtocolEnum.orbitals_and_eigenvalues, # Don't save wavefunctions in case there's a lot of errors
            keywords=keywords,
            scf_properties=properties,
            implicit_solvent= ddx_spec
        )

In [11]:
from qcelemental.models import DriverEnum

dataset = BasicDataset(
    dataset_name="OpenFF NAGL2 ESP Timing Benchmark ddx v1.0",
    dataset_tagline="PBE0/def2-TZVPPD/ddx water single point calculations of ~1000 diverse molecules.",
    description=(
        "PBE0/def2-TZVPPD/ddx water single point calculations of ~1000 diverse molecules sub-sampled from the ESP50k, multi-BR ESP, and I fragment datasets, to benchmark computational cost."
    ),
    driver=DriverEnum.properties, # This was DriverEnum.energy for the other ESP datasets, but I think to calculate dipole etc it needs to be properties.
    metadata=Metadata(
        submitter="amcisaac",
        long_description_url=(
            "https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2024-09-16-OpenFF-NAGL2-ESP-Timing-Benchmark-ddx-v1.0"
        )
    ),
    qc_specifications={
        "pbe0/def2-TZVPPD/ddx-water": ddx_qc_spec
            
    }
)

In [12]:
dataset._get_specifications()

{'pbe0/def2-TZVPPD/ddx-water': QCSpecification(program='psi4', driver=<SinglepointDriver.properties: 'properties'>, method='pbe0', basis='def2-tzvppd', keywords={'maxiter': 200, 'function_kwargs': {'properties': [<SCFProperties.Dipole: 'dipole'>, <SCFProperties.Quadrupole: 'quadrupole'>, <SCFProperties.LowdinCharges: 'lowdin_charges'>, <SCFProperties.MullikenCharges: 'mulliken_charges'>, <SCFProperties.MBISCharges: 'mbis_charges'>, <SCFProperties.MayerIndices: 'mayer_indices'>, <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>, <SCFProperties.DipolePolarizabilities: 'dipole_polarizabilities'>]}, 'dft_spherical_points': 590, 'dft_radial_points': 99, 'ddx': True, 'ddx_model': 'pcm', 'ddx_radii_scaling': 1.1, 'ddx_radii_set': 'uff', 'ddx_solvent_epsilon': 78.4, 'ddx_solvent': 'water'}, protocols=AtomicResultProtocols(wavefunction=<WavefunctionProtocolEnum.none: 'none'>, stdout=True, error_correction=ErrorCorrectionProtocol(default_policy=True, policies=None), native_files=<Nati

In [13]:
from collections import defaultdict

records_by_cmiles = defaultdict(list)
for record, molecule in br_records + esp50k_records + i_records:
    records_by_cmiles[
        molecule.to_smiles(isomeric=True, explicit_hydrogens=True, mapped=True)
    ].append((record, molecule))

In [14]:
from openff.qcsubmit.common_structures import MoleculeAttributes
import tqdm

for records in tqdm.tqdm(records_by_cmiles.values(), total=len(records_by_cmiles)):
    base_record, base_molecule = records[0]
    base_molecule._conformers = [m.conformers[0] for _, m in records]

    dataset.add_molecule(
        index=base_molecule.to_smiles(
            isomeric=True, explicit_hydrogens=False, mapped=False
        ),
        molecule=base_molecule,
        attributes=MoleculeAttributes.from_openff_molecule(base_molecule),
        extras=base_record.extras,
        keywords=base_record.to_qcschema_result().keywords,
    )

100%|███████████████████████████████████████████████████████████████| 1005/1005 [14:00<00:00,  1.20it/s]


# Exporting dataset

In [15]:
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file('dataset.smi', 'smi')
dataset.visualize("dataset.pdf", columns=8)

print(dataset.qc_specifications)

{'pbe0/def2-TZVPPD/ddx-water': QCSpec(method='pbe0', basis='def2-TZVPPD', program='psi4', spec_name='pbe0/def2-TZVPPD/ddx-water', spec_description='PBE0 functional with def2-TZVPPD basis set and DDX water (eps = 78.4) using UFF radii and 1.1x scaling factor', store_wavefunction=<WavefunctionProtocolEnum.none: 'none'>, implicit_solvent=DDXSettings(ddx_model='pcm', ddx_radii_scaling=1.1, ddx_radii_set='uff', ddx_solvent_epsilon=78.4, ddx_solvent='water'), maxiter=200, scf_properties=[<SCFProperties.Dipole: 'dipole'>, <SCFProperties.Quadrupole: 'quadrupole'>, <SCFProperties.LowdinCharges: 'lowdin_charges'>, <SCFProperties.MullikenCharges: 'mulliken_charges'>, <SCFProperties.MBISCharges: 'mbis_charges'>, <SCFProperties.MayerIndices: 'mayer_indices'>, <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>, <SCFProperties.DipolePolarizabilities: 'dipole_polarizabilities'>], keywords={'dft_spherical_points': 590, 'dft_radial_points': 99})}


# Dataset information

In [16]:
import numpy as np
from collections import Counter

In [17]:
print("n_molecules:", dataset.n_molecules)
print("n_conformers:", dataset.n_records)

n_molecules: 1005
n_conformers: 1009


In [18]:
n_confs = np.array(
    [mol.n_conformers for mol in dataset.molecules]
)
n_heavy_atoms = np.array(
    [mol.to_rdkit().GetNumHeavyAtoms() for mol in dataset.molecules]
)

In [19]:
print(
    "Number of conformers (min, mean, max):",
    n_confs.min(), n_confs.mean(), n_confs.max()
)
print("# heavy atoms")
counts = Counter(n_heavy_atoms)
for n_heavy in sorted(counts):
    print(f"{str(n_heavy):>3}: {counts[n_heavy]}")

Number of conformers (min, mean, max): 1 1.0039800995024875 2
# heavy atoms
  4: 3
  5: 4
  6: 23
  7: 46
  8: 121
  9: 180
 10: 180
 11: 230
 12: 218


In [20]:
from openff.units import unit
unique_charges = set([
    mol.total_charge.m_as(unit.elementary_charge)
    for mol in dataset.molecules
])
unique_charges

{-4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0}

In [21]:

masses = np.array([
    sum([atom.mass.m for atom in mol.atoms])
    for mol in dataset.molecules
])
print("MW (min, mean, max):", masses.min(), masses.mean(), masses.max())

MW (min, mean, max): 58.10263600000002 155.14636379830847 329.815488


In [22]:
elements = set(
    atom.symbol
    for mol in dataset.molecules
    for atom in mol.atoms
)
print(elements)

{'Br', 'F', 'I', 'N', 'Cl', 'C', 'P', 'H', 'S', 'O'}


In [23]:
print("## Metadata")
print(f"* Elements: {{{', '.join(dataset.metadata.dict()['elements'])}}}")

def print_field(od, field): print(f"\t* {field}: {od[field]}")

fields = ["basis", "implicit_solvent", "keywords", "maxiter", "method", "program"]
for spec, obj in dataset.qc_specifications.items():
    od = obj.dict()
    print("* Spec:", spec)
    for field in fields:
        print_field(od, field)
    print("\t* SCF properties:")
    for field in od["scf_properties"]:
        print(f"\t\t* {field}")

## Metadata
* Elements: {Br, F, I, N, Cl, C, P, H, S, O}
* Spec: pbe0/def2-TZVPPD/ddx-water
	* basis: def2-TZVPPD
	* implicit_solvent: {'ddx_model': 'pcm', 'ddx_radii_scaling': 1.1, 'ddx_radii_set': 'uff', 'ddx_solvent_epsilon': 78.4, 'ddx_solvent': 'water'}
	* keywords: {'dft_spherical_points': 590, 'dft_radial_points': 99}
	* maxiter: 200
	* method: pbe0
	* program: psi4
	* SCF properties:
		* dipole
		* quadrupole
		* lowdin_charges
		* mulliken_charges
		* mbis_charges
		* mayer_indices
		* wiberg_lowdin_indices
		* dipole_polarizabilities
